# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [15]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from citipy import citipy
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Import API key
from api_keys import geoapify_key

In [31]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,blackmans bay,-43.0167,147.3167,53.73,61,60,3.00,AU,1702585942
1,1,anadyr,64.7500,177.4833,16.30,86,100,24.61,RU,1702585942
2,2,tolanaro,-25.0319,46.9987,78.87,83,20,20.71,MG,1702585943
3,3,newman,37.3138,-121.0208,63.23,60,100,1.99,US,1702585943
4,4,holualoa,19.6228,-155.9522,76.84,65,40,26.46,US,1702585943


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [17]:
# Configure the map plot
city_map = city_data_df.hvplot.points("Lng", 
                                      "Lat", 
                                      geo = True,
                                      color = "City",
                                      alpha = 0.6,
                                      size = "Humidity",
                                      tiles = "OSM",
                                      width = 1400, 
                                      height = 800
                                     )

# Display the map
city_map


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [37]:
# Narrow down cities that fit criteria and drop any results with null values
filter_cities = city_data_df.loc[
    (city_data_df["Max Temp"] < 80) & 
    (city_data_df["Max Temp"] > 70) & 
    (city_data_df["Wind Speed"] < 5) & 
    (city_data_df["Cloudiness"] < 15)]

# Drop any rows with null values
filter_cities = filter_cities.dropna()

# Display sample data
filter_cities

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
42,42,saint-pierre,-21.3393,55.4781,76.68,78,0,4.61,RE,1702585956
181,181,orerokpe,5.6351,5.8896,76.87,63,11,3.71,NG,1702585981
389,389,gangolli,13.6500,74.6667,74.50,59,3,4.99,IN,1702586031
406,406,yilan,24.7570,121.7530,75.02,86,2,3.56,TW,1702586034
464,464,oasis,33.4659,-116.0989,76.33,25,0,1.01,US,1702586043
475,475,kone,-21.0595,164.8658,79.54,59,13,2.64,NC,1702586047
510,510,mangrol,21.1167,70.1167,73.89,68,3,1.97,IN,1702586055
530,530,yeppoon,-23.1333,150.7333,73.18,69,0,4.29,AU,1702586061


### Step 3: Create a new DataFrame called `hotel_df`.

In [38]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filter_cities[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df


,City,Country,Lat,Lng,Humidity,Hotel Name
42,saint-pierre,RE,-21.3393,55.4781,78,
181,orerokpe,NG,5.6351,5.8896,63,
389,gangolli,IN,13.6500,74.6667,59,
406,yilan,TW,24.7570,121.7530,86,
464,oasis,US,33.4659,-116.0989,25,
475,kone,NC,-21.0595,164.8658,59,
510,mangrol,IN,21.1167,70.1167,68,
530,yeppoon,AU,-23.1333,150.7333,69,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [39]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
          "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row["Lng"]
    latitude = row["Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = "circle:{},{},{}".format(longitude, latitude, radius)
    params["bias"] = "proximity:{},{}".format(longitude, latitude)
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
saint-pierre - nearest hotel: Tropic Hotel
orerokpe - nearest hotel: No hotel found
gangolli - nearest hotel: Ganesh Hotel (Nanda)
yilan - nearest hotel: 蘭城晶英酒店(6-11F)
oasis - nearest hotel: No hotel found
kone - nearest hotel: Pacifik Appartels
mangrol - nearest hotel: No hotel found
yeppoon - nearest hotel: Blossom's on Seaspray


,City,Country,Lat,Lng,Humidity,Hotel Name
42,saint-pierre,RE,-21.3393,55.4781,78,Tropic Hotel
181,orerokpe,NG,5.6351,5.8896,63,No hotel found
389,gangolli,IN,13.6500,74.6667,59,Ganesh Hotel (Nanda)
406,yilan,TW,24.7570,121.7530,86,蘭城晶英酒店(6-11F)
464,oasis,US,33.4659,-116.0989,25,No hotel found
475,kone,NC,-21.0595,164.8658,59,Pacifik Appartels
510,mangrol,IN,21.1167,70.1167,68,No hotel found
530,yeppoon,AU,-23.1333,150.7333,69,Blossom's on Seaspray


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [41]:
# Configure the map plot
hotel_map = hotel_df.hvplot.points("Lng", 
                                   "Lat", 
                                   geo = True,
                                   color = "City",
                                   size = "Humidity",
                                   tiles = "OSM",
                                   hover_cols = ["Hotel Name", "Country"],
                                   xlim = (-180, 180),
                                   ylim = (-90,90),
                                   width = 800,
                                   height = 600
                                  )


# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)